# Modeling of relative Yield, P-Uptake and P-Balance

Lukas Graz  
February 13, 2025

In [ ]:
RES <- readRDS("data/RES.rds")
D <- RES$D
lgr::get_logger("mlr3")$set_threshold("warn")
# d <- RES$data


## Setup

In [ ]:
library(mlr3verse, quietly = TRUE)

mse <- msrs(c("regr.mse"))

if (!interactive())
  lgr::get_logger("mlr3")$set_threshold("warn")

get_benchi_table <- function(tasks, nfolds = 5, resamplings = NULL) {
  # TODO activate xgboost and ranger for == uncomment things below
  set.seed(123)
  learners <- lrns(c("regr.featureless", "regr.lm"
    # , "regr.xgboost", "regr.ranger"
    ))
  # learners$regr.xgboost$param_set$set_values(
  #   eta = 0.04, 
  #   nrounds = 300, 
  #   max_depth = 2
  # )

  benchi <- xfun::cache_rds({
    benchmark(benchmark_grid(
      tasks, 
      learners, 
      if(is.null(resamplings)) rsmp("cv", folds = nfolds) else resamplings
    ))
  }, 
  file = "benchmark.rds", 
  dir = "cache/",
  hash = list(tasks, nfolds)
  )
  
  res <- tidyr::pivot_wider(benchi$aggregate(mse), 
    id_cols = task_id,
    names_from = learner_id,
    values_from = regr.mse
  ) |> as.data.frame()
  
  rownames(res) <- res$task_id
  res <- res[, -1]
  colnames(res) <- gsub("regr.", "", colnames(res))
  stopifnot(any(colnames(res) == "featureless"))
  res <- 1 - res / res$featureless
  res[, -1, drop = FALSE] |> round(3)
}


Testing prediction quality using

-   Linear models
-   Random forests (default parameters)
-   XGBoost (with parameter tuning)

**Weather Variables:**

In [ ]:
Weather_vars <- c(
  "anavg_temp", "ansum_prec",
  "juvdev_prec", "juvdev_sun",
  "ansum_sun", "juvdev_temp"
)

# set NA's to 0 but include a column to indicate that
NA_weather <- is.na(D$juvdev_sun)
D[NA_weather, Weather_vars] <- 0
D$NA_weather <- NA_weather
Weather_vars <- c(Weather_vars, "NA_weather")

stopifnot(all(Weather_vars %in% names(D)))
Weather_vars


[1] "anavg_temp"  "ansum_prec"  "juvdev_prec" "juvdev_sun"  "ansum_sun"  
[6] "juvdev_temp" "NA_weather" 

In [ ]:
P_var_sets <- list(
  onlyweather = NULL,
  k = "k",
  PS = "PS_log",
  kPS = c("PS_log", "k", "kPS_log"),
  AAE10 = "P_AAE10_log",
  CO2 = "P_CO2_log",
  AAE10_CO2 = c("P_AAE10_log", "P_CO2_log", "P_AAE10_CO2_loglog"),
  AAE10_CO2_kPS = c("P_AAE10_log", "P_CO2_log", "PS_log", "k", "kPS_log"),
  CO2_kPS = c("P_CO2_log", "PS_log", "k", "kPS_log")
)
Earth_vars <- c(
  "soil_0_20_clay", "soil_0_20_pH_H2O", "soil_0_20_Corg", "soil_0_20_silt"
)


In [ ]:
Y_vars_yield <- c("Ymain_norm", "Ymain_rel", "annual_P_uptake", "annual_P_balance")

Y_vars_earth <- c("PS_log", "k", "kPS_log", "P_AAE10_log", "P_CO2_log")


impute NA’s for Earth variables. impute value with the mean of Site X block

In [ ]:
# xtabs(~is.na(D$soil_0_20_silt) + Site + block, D)
for (var in Earth_vars) {
  D[[var]] <- ave(D[[var]], D$Site, D$block, FUN = \(x) ifelse(is.na(x), mean(x, na.rm = TRUE), x))
}


remove Cadenazzoo since too many crucial data missing

In [ ]:
D <- D[D$site != "CAD", ]
D$Site <- droplevels(D$Site)


Now check NA’s

In [ ]:
sapply(D[, unique(c(Y_vars_yield, Y_vars_earth, Weather_vars, unlist(P_var_sets), Earth_vars))], 
  \(x) sum(is.na(x)|is.nan(x))) |> cbind()


                   [,1]
Ymain_norm           23
Ymain_rel           117
annual_P_uptake      70
annual_P_balance     55
PS_log                0
k                     0
kPS_log               0
P_AAE10_log           0
P_CO2_log             0
anavg_temp            0
ansum_prec            0
juvdev_prec           0
juvdev_sun            0
ansum_sun             0
juvdev_temp           0
NA_weather            0
P_AAE10_CO2_loglog    0
soil_0_20_clay        0
soil_0_20_pH_H2O      0
soil_0_20_Corg        0
soil_0_20_silt        0

## Predicting Yield variables (with/without Weather data) with Earth-dynamics

### With Weather data

In [ ]:
P_var_sets


$onlyweather
NULL

$k
[1] "k"

$PS
[1] "PS_log"

$kPS
[1] "PS_log"  "k"       "kPS_log"

$AAE10
[1] "P_AAE10_log"

$CO2
[1] "P_CO2_log"

$AAE10_CO2
[1] "P_AAE10_log"        "P_CO2_log"          "P_AAE10_CO2_loglog"

$AAE10_CO2_kPS
[1] "P_AAE10_log" "P_CO2_log"   "PS_log"      "k"           "kPS_log"    

$CO2_kPS
[1] "P_CO2_log" "PS_log"    "k"         "kPS_log"  

Algorithm learns to predict location from weather since we do not do stratified cross-validation (leaving out locations).

In [ ]:
Tables_yield_weather <- list()
for(yvar in Y_vars_yield){
  ind <- complete.cases(D[,unique(c(yvar ,Weather_vars, unlist(P_var_sets)))])

  mytsks <- list()
  for (nam in names(P_var_sets)) {
    mytsk <- as_task_regr(
      D[ind, c(yvar, Weather_vars, P_var_sets[[nam]])],
      target = yvar,
      id = nam)
    mytsks[[nam]] <- mytsk
  }
  Tables_yield_weather[[yvar]] <- get_benchi_table(mytsks)
}

Tables_yield_weather


$Ymain_norm
                  lm
onlyweather   -0.018
k              0.023
PS             0.204
kPS            0.203
AAE10          0.147
CO2            0.231
AAE10_CO2      0.235
AAE10_CO2_kPS  0.220
CO2_kPS        0.246

$Ymain_rel
                 lm
onlyweather   0.132
k             0.105
PS            0.184
kPS           0.189
AAE10         0.222
CO2           0.210
AAE10_CO2     0.245
AAE10_CO2_kPS 0.238
CO2_kPS       0.157

$annual_P_uptake
                 lm
onlyweather   0.391
k             0.353
PS            0.446
kPS           0.415
AAE10         0.496
CO2           0.463
AAE10_CO2     0.491
AAE10_CO2_kPS 0.463
CO2_kPS       0.449

$annual_P_balance
                 lm
onlyweather   0.113
k             0.077
PS            0.627
kPS           0.632
AAE10         0.432
CO2           0.594
AAE10_CO2     0.591
AAE10_CO2_kPS 0.652
CO2_kPS       0.644

In [ ]:
tmp <- do.call(cbind, lapply(Tables_yield_weather, \(x) x$lm)); 
rownames(tmp) <- rownames(Tables_yield_weather[[1]])
(Tables_yield_weather_lm <- tmp)


              Ymain_norm Ymain_rel annual_P_uptake annual_P_balance
onlyweather       -0.018     0.132           0.391            0.113
k                  0.023     0.105           0.353            0.077
PS                 0.204     0.184           0.446            0.627
kPS                0.203     0.189           0.415            0.632
AAE10              0.147     0.222           0.496            0.432
CO2                0.231     0.210           0.463            0.594
AAE10_CO2          0.235     0.245           0.491            0.591
AAE10_CO2_kPS      0.220     0.238           0.463            0.652
CO2_kPS            0.246     0.157           0.449            0.644

### Without Weather data

In [ ]:
P_var_sets_noweather <- P_var_sets[-1]

Tables_yield <- list()
for(yvar in Y_vars_yield){
  ind <- complete.cases(D[,unique(c(yvar, unlist(P_var_sets_noweather)))])

  mytsks <- list()
  for (nam in names(P_var_sets_noweather)) {
    mytsk <- as_task_regr(
      D[ind,
        c(yvar, P_var_sets_noweather[[nam]]
        # ,"Site"
        )],
      target = yvar,
      id = nam)
    # mytsk$set_col_roles("Site", "group")
    mytsks[[nam]] <- mytsk
  }
  Tables_yield[[yvar]] <- get_benchi_table(mytsks)
}

Tables_yield


$Ymain_norm
                 lm
k             0.021
PS            0.223
kPS           0.215
AAE10         0.109
CO2           0.228
AAE10_CO2     0.232
AAE10_CO2_kPS 0.233
CO2_kPS       0.232

$Ymain_rel
                  lm
k             -0.009
PS             0.051
kPS            0.036
AAE10          0.152
CO2            0.105
AAE10_CO2      0.129
AAE10_CO2_kPS  0.139
CO2_kPS        0.094

$annual_P_uptake
                  lm
k             -0.005
PS             0.034
kPS            0.030
AAE10          0.095
CO2            0.075
AAE10_CO2      0.097
AAE10_CO2_kPS  0.049
CO2_kPS        0.046

$annual_P_balance
                 lm
k             0.018
PS            0.553
kPS           0.540
AAE10         0.277
CO2           0.500
AAE10_CO2     0.491
AAE10_CO2_kPS 0.562
CO2_kPS       0.561

In [ ]:
tmp <- do.call(cbind, lapply(Tables_yield, \(x) x$lm));
rownames(tmp) <- rownames(Tables_yield[[1]])
(Tables_yield_lm <- tmp)


              Ymain_norm Ymain_rel annual_P_uptake annual_P_balance
k                  0.021    -0.009          -0.005            0.018
PS                 0.223     0.051           0.034            0.553
kPS                0.215     0.036           0.030            0.540
AAE10              0.109     0.152           0.095            0.277
CO2                0.228     0.105           0.075            0.500
AAE10_CO2          0.232     0.129           0.097            0.491
AAE10_CO2_kPS      0.233     0.139           0.049            0.562
CO2_kPS            0.232     0.094           0.046            0.561

xgboost & ranger are no good in this setting since only very few variables available

## Predicting Earth-dynamics via soil variables (with/without Treatment)

In [ ]:
Earth_vars


[1] "soil_0_20_clay"   "soil_0_20_pH_H2O" "soil_0_20_Corg"   "soil_0_20_silt"  

[1] "PS_log"      "k"           "kPS_log"     "P_AAE10_log" "P_CO2_log"  

In [ ]:
D$is.trt0 <- D$Treatment == "P0"
D$is.trt1 <- D$Treatment == "P100"

mytsks_treatment <- list()
mytsks_notreatment <- list()
for(yvar in Y_vars_earth){
  ind <- complete.cases(D[,unique(c(yvar, unlist(P_var_sets_noweather)))])

    mytsk <- as_task_regr(
      D[ind, c(yvar, Earth_vars, "is.trt1", "is.trt0"
        # ,"Site"
        )],
      target = yvar,
      id = yvar)
    # mytsk$set_col_roles("Site", "group")
    mytsks_treatment[[yvar]] <- mytsk

    mytsk <- as_task_regr(
      D[ind, c(yvar, Earth_vars
        # ,"Site"
        )],
      target = yvar,
      id = yvar)
    # mytsk$set_col_roles("Site", "group")
    mytsks_notreatment[[yvar]] <- mytsk    
}

Tables_earth_treatment <- get_benchi_table(mytsks_treatment)
Tables_earth_notreatment <- get_benchi_table(mytsks_notreatment)


In [ ]:
Tables_earth_treatment


               lm
PS_log      0.885
k           0.328
kPS_log     0.752
P_AAE10_log 0.824
P_CO2_log   0.792

               lm
PS_log      0.042
k           0.267
kPS_log     0.023
P_AAE10_log 0.365
P_CO2_log   0.025

In [ ]:
tmp <- rbind(
  "soil + treatment" = Tables_earth_treatment[["lm"]],
  "only soil" = Tables_earth_notreatment[["lm"]]
)
colnames(tmp) <- rownames(Tables_earth_treatment)
(Tables_earth_lm <- tmp)


                 PS_log     k kPS_log P_AAE10_log P_CO2_log
soil + treatment  0.885 0.328   0.752       0.824     0.792
only soil         0.042 0.267   0.023       0.365     0.025

In [ ]:
saveRDS(list(
  Tables_yield = Tables_yield,
  Tables_yield_weather = Tables_yield_weather,
  Tables_earth_treatment = Tables_earth_treatment,
  Tables_earth_notreatment = Tables_earth_notreatment,
  Tables_yield_lm = Tables_yield_lm,
  Tables_yield_weather_lm = Tables_yield_weather_lm,
  Tables_earth_lm = Tables_earth_lm
), "cache/benchmark-tables.rds")


------------------------------------------------------------------------

In [ ]:
cor(D$annual_P_balance, D$PS) # 0.54389


[1] NA

[1] 0.6449276

[1] NA

We did manage to have high predictive power for weather. This could also be due to our regression models recovering location&year from it and hence still overfitting on the test set.

Without Weather data we only managed for annual balance to get some predictive power (30%). Since we the balance is uptake - fert_P, this means that we mostly predicted fert_P. Interestingly PS is best to predict this quantity

### Legacy Code

In [ ]:

# Get parameter estimates for XGBoost
t <- as_task_regr(
  subset(D[complete.cases(D$annual_P_balance),], 
    select = c("annual_P_balance", P_var_sets$AAE10_CO2_kPS#, Weather_vars
    )),
  target = "annual_P_balance"
)

l <- lrn("regr.xgboost",
  nrounds = 500  # More iterations due to lower learning rate
)

# Create search space
ps <- ps(
  max_depth = p_int(2, 4),
  eta = p_dbl(0.001, 0.3, tags = "logscale")
)

# Setup tuning
instance <- ti(
  task = t,
  learner = l,
  resampling = rsmp("cv", folds = 3),
  measure = msr("regr.mse"),
  terminator = trm("none"),
  search_space = ps
)

# Grid search
tuner <- mlr3tuning::tnr("grid_search")
tuner$optimize(instance)
instance$result


Ymain_rel max_depth eta learner_param_vals x_domain regr.mse <int> <num> <list> <list> <num> 1: 2 0.067444 \<list\[5\]\> \<list\[2\]\> 177.18

P uptake max_depth eta learner_param_vals x_domain regr.mse <int> <num> <list> <list> <num> 1: 2 0.034222 \<list\[5\]\> \<list\[2\]\> 137.41

annual_P_balance max_depth eta learner_param_vals x_domain regr.mse <int> <num> <list> <list> <num> 1: 2 0.034222 \<list\[5\]\> \<list\[2\]\> 145.21

In [ ]:
# nlme.coef$kPS_log <- nlme.coef$k * nlme.coef$PS
# 
# 
# nlme.coef.mrg <- merge(nlme.coef,allP[allP$year>=2017,],by = "uid")
# # add log-transformed versions
# D$kPS_log <- log(D$kPS_log)
# D$PS_log <- log(D$PS)
# D$soil_0_20_P_AAE10_log <- log(D$soil_0_20_P_AAE10)
# D$soil_0_20_P_CO2_log <- log(D$soil_0_20_P_CO2)
# 
# D$k



subset(D, select = c("Ymain_rel", P_var_sets$AAE10_CO2_kPS, Weather_vars))


    Ymain_rel P_AAE10_log   P_CO2_log     PS_log          k   kPS_log
55      72.33   1.8870696 -1.46967597 -2.9145147 0.20191085 -4.514444
56      92.91   1.8870696 -1.46967597 -2.9145147 0.20191085 -4.514444
57      65.60   1.8870696 -1.46967597 -2.9145147 0.20191085 -4.514444
58         NA   1.8870696 -1.46967597 -2.9145147 0.20191085 -4.514444
59         NA   1.8870696 -1.46967597 -2.9145147 0.20191085 -4.514444
60         NA   1.8870696 -1.46967597 -2.9145147 0.20191085 -4.514444
61      84.81   1.8870696 -1.46967597 -2.9145147 0.20191085 -4.514444
62         NA   1.8245493 -1.34707365 -2.6264148 0.23310593 -4.082677
63     106.32   1.8245493 -1.34707365 -2.6264148 0.23310593 -4.082677
64      69.17   1.8245493 -1.34707365 -2.6264148 0.23310593 -4.082677
65         NA   1.8245493 -1.34707365 -2.6264148 0.23310593 -4.082677
66      84.97   1.8245493 -1.34707365 -2.6264148 0.23310593 -4.082677
67      82.00   1.8245493 -1.34707365 -2.6264148 0.23310593 -4.082677
68         NA   1.82

# Methods

we used machine learning methods to assess how much information different sets of variables (c.f. `P_var_sets`) have each on the dependent variable (Puptake, Y-rel, P-balance), how redundant this information is. The machine learning methods to quantify the predictive power of different variable sets are: i) ordinary least squares (OLS) as a baseline; ii) XGBoost (gradient boosting with tree-based models and hyperparameter tuning for learning rate and tree depth) (arxiv:1603.02754); iii) Random Forests (with default parameters) (doi:10.1023/A:1010933404324). Computations were performed using the mlr3 framework (doi:10.21105/joss.01903). Performance was measured as percentage of explained variance on hold-out data via 5-fold cross-validation, calculated as (1 - MSE/Variance(y)), where MSE represents mean squared error.

We tried adjusting for weather variables but it seems that the ML-methods rather reconstruct the site-specific patterns….